In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image


model=models.vgg19(pretrained=True).features
device=torch.device( "cuda" if (torch.cuda.is_available()) else 'cpu')


d:\Home price\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Home price\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
def image_loader(path):
    image=Image.open(path)
    transform=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    image=transform(image).unsqueeze(0)
    return image.to(device,torch.float)

#Path delle immagini di contenuto e stile
content_image=image_loader('D:\\NTS\\data\\Timothee.jpg')
style_image=image_loader('D:\\NTS\\data\\Notte stellata.jpg')

#Generazione dell'immagine generata
generated_image=content_image.clone().requires_grad_(True)

class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        #layer convoluzionali estratti dal modello VGG
        self.req_features= ['0',
                            '5',
                            '10',
                            '19',
                            '28'
                            ]
        
        self.model=models.vgg19(pretrained=True).features[:29]


    def forward(self,x):
        features=[]
    
        for layer_num,layer in enumerate(self.model):
            x=layer(x)
            if (str(layer_num) in self.req_features):
                features.append(x)
        return features

def calc_content_loss(gen_feat,orig_feat):
    #calcolo della loss per il contenuto
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

def calc_style_loss(gen,style):
    B,C,H,W=gen.shape
    #calcolo della loss e della matrice di Gram per lo stile
    G=torch.mm(gen.view(C,H*W),gen.view(C,H*W).t())
    A=torch.mm(style.view(C,H*W),style.view(C,H*W).t())

    style_l=torch.mean((G-A)**2)
    return style_l

def calculate_loss(gen_features, orig_features, style_features):
    style_loss=content_loss=0
    i=[0,1,2,3,4]
    wl = [1, 0.75, 0.2, 0.2, 0.2]
    #wl = [1/5, 1/5, 1/5, 1/5, 1/5] pesi tutti uguali usati per visualizzare la style image
    #wl = [0.1, 0.1, 0.1, 0.1, 10]  pesi usati epr enfatizzare l'ultimo livello

    for i,gen,cont,style in zip(i,gen_features,orig_features,style_features):
        cont=cont.detach()    
        style=style.detach()
        if(i==3):
            content_loss+=calc_content_loss(gen,cont)

        style_loss+=wl[i]*calc_style_loss(gen,style)

    #calcolo della loss totale
    total_loss=alpha*content_loss + beta*style_loss
    return total_loss


model=VGG().to(device).eval()

#Inizializzazione dei parametri per il modello
epoch=200
lr=0.02
alpha=1
beta=50000

optimizer=optim.Adam([generated_image],lr=lr)

content_feautes=model(content_image)
style_featues=model(style_image)


for e in range (epoch):

    gen_features=model(generated_image)

    # calcolo della loss totale
    total_loss=calculate_loss(gen_features, content_feautes, style_featues)


    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    #print(e)

    if(e==0) :
      print("Start")

    if(e % 100 == 0):
        print(f'process perchentage: {e/epoch*100}%, Loss:{total_loss} ')
        save_image(generated_image,"out.png")

    if(e==epoch-1) :
        print("end")

d:\Home price\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Home price\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start
process perchentage: 0.0%, Loss:4710507479040.0 
process perchentage: 50.0%, Loss:10252949504.0 
end
